# NB1. Network Statistics

***Samantha Castro, Christopher Cumi, Juan Fernández, and Juliana Ramayo***

Consider the following networks:

* **Facebook Northwester University**(socfb-Northwestern25.edges.gz). Network of Facebook users at Northwestern University. Nodes represent people, and links stand for Facebook friend connections.
* **US air transportation** (openflights_usa.edges.gz). The US air transportation network using flight data from OpenFlights.org. Nodes represent airports, and links stand for connections between them.
* **Twitter USA Politics**(retweet-digraph.edges.gz). Retweet directed network with weigtht on Twitter, among people sharing posts about US politics. Links represent retweets of posts that used different hashtags (#tcot, #p2). The direction of the link from user A to B indicates that a message has propagated from A to B.

In [ ]:
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Task 1
Create a table including the following characteristics for each network:
* Number of Nodes $N$.
* Number of Links $L$.
* Density $d$.
* Average Degree $\langle k\rangle $.
* Clustering Coefficient $C_C$.
    
Consider the following observations:
* In the case of undirected networks, compute the average in-degree.
* In the case of undirected networks, compute the clustering coefficient without taking into account the directions of the edges. In NetworkX it is possible to use the ``` G.to_undirected() ``` method to return an undirected copy of a graph G.


In [ ]:
# Helper function to compute network statistics
def network_stats(G):
  """
  Calculates the statistical measures of a network graph.

  Parameters:
  - G (networkx.Graph or networkx.DiGraph)

  Returns:
  - dict: A dictionary containing the statistics
  """

  # Get number of nodes, edges and density from the graph
  num_nodes = len(G.nodes)
  num_links = len(G.edges)
  density = nx.density(G)

  if G.is_directed():
    # Compute average degree for directed graph
    avg_degree = num_links / float(num_nodes)
    # Calculate the average clustering coefficient for directed graphs
    clustering_coeff = sum(nx.clustering(G).values()) / len(nx.clustering(G))
  else:
    # Compute average degree for undirected graph
    avg_degree = 2 * (num_links) / float(num_nodes)
    # Calculate the average clustering coefficient for undirected graphs
    clustering_coeff = nx.average_clustering(G)

  return {
      "Number of Nodes": num_nodes,
      "Number of Links": num_links,
      "Density": density,
      "Average Degree": avg_degree,
      "Clustering Coefficient": clustering_coeff
      }

In [ ]:
# Load the networks from the provided files
facebook = nx.read_edgelist("/content/socfb-Northwestern25.edgelist", create_using=nx.Graph())  # Undirected graph
transportation = nx.read_edgelist("/content/openflights_usa.edges", create_using=nx.Graph())  # Directed graph
twitter = nx.read_weighted_edgelist("/content/retweet-digraph.edges", create_using=nx.DiGraph(), delimiter=' ')  # Directed graph with weights

# Calculate stats for each network
stats_facebook = network_stats(facebook)
stats_transportation = network_stats(transportation)
stats_twitter = network_stats(twitter)

# Create a DataFrame to display the statistics of each network
network_df = pd.DataFrame([stats_facebook, stats_transportation, stats_twitter], index=["Facebook Northwester University", "US Air Transportation", "Twitter USA Politics"])
network_df

,Number of Nodes,Number of Links,Density,Average Degree,Clustering Coefficient
Facebook Northwester University,10567,488337,0.008748,92.426800,0.237991
US Air Transportation,546,2781,0.018691,10.186813,0.493045
Twitter USA Politics,18470,48365,0.000142,2.618571,0.014210


The average shortest-path length is a common aggregate distance measure for Networks. It can be obtained by averaging the shortest-path lengths across all pairs of nodes. The definition of this distance-based measure assume that the shortest-path length is defined for each pair of nodes. If there is any pairs without a path, then the the average path length is not defined. One way to present this result is by measuring only on the giant component; for the directed network it is possible to consider directed paths in the giants strongly connected component. However, due to the number of possible pairs of nodes, the computing of the average shortest-path length can be computational extensive.

## Task 2
Create a function ``` average_path_length_sample(G, N_sample)``` to compute the average path length on a Network. The function must identify if the network is directed or not.  The following method can be useful: ``` G.is_directed()```. In the case of directed networks it should use the strongly connected component to compute it. On the other hand, if the network is undirected, it should use the giang connected component of the network.

In order to compute the average path length on a sample. Make a sample of ```N_sample``` randomly chosen nodes on the connected component and compute the average path length using it.

The function must input ```G```a Network and ```N_sample```the number of nodes to be considered in the sample and output the average path length.

Compute the average path length of the three given networks and add them into the table using ```N_sample=1000```.

In [ ]:
import random

def average_path_length_sample(G, N_sample=1000):
  """
  Estimate the average path length on a network using a sample of nodes.

  Parameters:
  - G (networkx.Graph or networkx.DiGraph): The network to be analyzed
  - N_sample (int): The number of nodes to be considered in the sample. Default is 1000.

  Returns:
  - float: The average path length calculated using the sample.
  """
  # Check if the graph is directed
  if G.is_directed():
    # Find the largest strongly connected component
    largest_cc = max(nx.strongly_connected_components(G), key=len)
  else:
    # Find the largest connected component
    largest_cc = max(nx.connected_components(G), key=len)

  # Create a subgraph containing only the nodes in the largest connected component
  subgraph = G.subgraph(largest_cc)

  # Sample set of nodes from the subgraph
  sampled_nodes = random.sample(list(subgraph.nodes), min(N_sample, len(subgraph)))
  # Calculate the shortest path lengths from each sampled node to all other nodes
  lengths = []
  for node in sampled_nodes:
    lengths.extend(nx.single_source_shortest_path_length(subgraph, node).values())
  # Compute the average of these path lengths
  return sum(lengths) / len(lengths)

In [ ]:
# Compute average path lengths
avg_path_length_facebook = average_path_length_sample(facebook)
avg_path_length_transportation = average_path_length_sample(transportation)
avg_path_length_twitter = average_path_length_sample(twitter)

# Print results
print("Average Path Lengths:")
print("Facebook Northwester University:", avg_path_length_facebook)
print("US Air Transportation:", avg_path_length_transportation)
print("Twitter USA Politics:", avg_path_length_twitter)

Average Path Lengths:
Facebook Northwester University: 2.714936699250261
US Air Transportation: 3.1914250604947663
Twitter USA Politics: 5.562612903225807


In [ ]:
# Update the DataFrame with the results
network_df['Average Path Length'] = [avg_path_length_facebook, avg_path_length_transportation, avg_path_length_twitter]
network_df

,Number of Nodes,Number of Links,Density,Average Degree,Clustering Coefficient,Average Path Length
Facebook Northwester University,10567,488337,0.008748,92.426800,0.237991,2.714937
US Air Transportation,546,2781,0.018691,10.186813,0.493045,3.191425
Twitter USA Politics,18470,48365,0.000142,2.618571,0.014210,5.562613


## Useful NetworkX Methods

* [Reading and writing graphs](https://networkx.github.io/documentation/networkx-1.9/reference/readwrite.html). Check the ```read_edgelist``` method.
* [Components](https://networkx.github.io/documentation/stable/reference/algorithms/component.html).

## References
[1] F. Mencszer, S. Fortunato, C. A. Davis (2020). A First Course in Network Science.